# **IP4_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:
1. S_target = 5500 (sales target)
2. H_FT_target = 800 (work hours target for full-time employees)
3. H_PT_target = 320 (work hours target for part-time employees)
4. O_FT_target = 100 (overtime hours target for full-time employees)
5. r_FT = 5 (average records sold per hour by a full-time employee)
6. r_PT = 2 (average records sold per hour by a part-time employee)
7. P1, P2, P3, P4 = prioroty factors with P1 > P2 > P3 > P4
8. F = 2 (concern factor for full-time employee utilization)
9. C = 3 (relative cost of overtime for part-time employees)

Decision Variables:
1. S_Dev = deviation from sales target
2. H_Dev_FT = deviation from work hours target for full-time employees
3. H_Dev_PT = deviation from work hours target for part-time employees
4. O_Dev_FT = deviation from overtime hours target for full-time employees
5. O_Dev_PT = deviation from overtime hours without limit for part-time employees

Objective Function:
Minimize O = P1*S_Dev + P2*O_Dev_FT + P3*F*H_Dev_FT + P3*

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here is the Pyomo code: All data is as defined in your problem.

```python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# sample data
P1 = 5
P2 = 4
P3 = 3
P4 = 2
r_FT = 5
r_PT = 2
F = 2
C = 3
S_target = 5500
H_FT_target = 800
H_PT_target = 320
O_FT_target = 100

# Define Variables
model.S_Dev = Var(within=NonNegativeReals)
model.H_Dev_FT = Var(within=NonNegativeReals)
model.H_Dev_PT = Var(within=NonNegativeReals)
model.O_Dev_FT = Var(within=NonNegativeReals)
model.O_Dev_PT = Var(within=NonNegativeReals)

model.H_FT_Hours = Var(within=NonNegativeReals)
model.H_PT_Hours = Var(within=NonNegativeReals)
model.O_FT_Hours = Var(within=NonNegativeReals)
model.O_PT_Hours = Var(within=NonNegativeReals)

# Define Objective
model.obj = Objective(expr = P1*model.S_Dev + P2*model.O_Dev_FT + P3*F*model.H_Dev_FT + 
                      P3*model.H_Dev_PT + P4*model.O_Dev_PT, sense=minimize)

# Define Constraints
model.sales = Constraint(expr= S_target - model.S_Dev <= r_FT*model.H_FT_Hours + 
                          r_PT*model.H_PT_Hours <= S_target + model.S_Dev)

model.full_time_hours = Constraint(expr= H_FT_target - model.H_Dev_FT <= model.H_FT_Hours 
                                   <= H_FT_target + model.H_Dev_FT)

model.part_time_hours = Constraint(expr= H_PT_target - model.H_Dev_PT <= model.H_PT_Hours 
                                   <= H_PT_target + model.H_Dev_PT)

model.full_time_overtime = Constraint(expr= O_FT_target - model.O_Dev_FT <= model.O_FT_Hours 
                                       <= O_FT_target + model.O_Dev_FT)

model.part_time_overtime = Constraint(expr = model.O_Dev_PT >= model.O_PT_Hours)

# Solve
SolverFactory('glpk').solve(model)

# Display solution
model.pprint()

```

Please note that this code does not consider overtime hours without limit for part-time employees. Also, you would need to adjust the objective function and constraints according to the actual problem requirements.

Replace the sample data with actual data when you use this code.

In the constraints, <= operator is used which indicates that either side of the inequality can be equal to or less than the other side. If you want strict inequality, that is not directly support in LP/MIP models.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# sample data
P1 = 5
P2 = 4
P3 = 3
P4 = 2
r_FT = 5
r_PT = 2
F = 2
C = 3
S_target = 5500
H_FT_target = 800
H_PT_target = 320
O_FT_target = 100

# Define Variables
model.S_Dev = Var(within=NonNegativeReals)
model.H_Dev_FT = Var(within=NonNegativeReals)
model.H_Dev_PT = Var(within=NonNegativeReals)
model.O_Dev_FT = Var(within=NonNegativeReals)
model.O_Dev_PT = Var(within=NonNegativeReals)

model.H_FT_Hours = Var(within=NonNegativeReals)
model.H_PT_Hours = Var(within=NonNegativeReals)
model.O_FT_Hours = Var(within=NonNegativeReals)
model.O_PT_Hours = Var(within=NonNegativeReals)

# Define Objective
model.obj = Objective(expr = P1*model.S_Dev + P2*model.O_Dev_FT + P3*F*model.H_Dev_FT +
                      P3*model.H_Dev_PT + P4*model.O_Dev_PT, sense=minimize)

# Define Constraints
model.sales = Constraint(expr= S_target - model.S_Dev <= r_FT*model.H_FT_Hours +
                          r_PT*model.H_PT_Hours <= S_target + model.S_Dev)

model.full_time_hours = Constraint(expr= H_FT_target - model.H_Dev_FT <= model.H_FT_Hours
                                   <= H_FT_target + model.H_Dev_FT)

model.part_time_hours = Constraint(expr= H_PT_target - model.H_Dev_PT <= model.H_PT_Hours
                                   <= H_PT_target + model.H_Dev_PT)

model.full_time_overtime = Constraint(expr= O_FT_target - model.O_Dev_FT <= model.O_FT_Hours
                                       <= O_FT_target + model.O_Dev_FT)

model.part_time_overtime = Constraint(expr = model.O_Dev_PT >= model.O_PT_Hours)

# Solve
SolverFactory('glpk').solve(model)

# Display solution
model.pprint()

NameError: name 'model' is not defined

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [3]:
from pyomo.environ import *

# sample data
P1 = 4
P2 = 3
P3 = 2
P4 = 1
r_FT = 5
r_PT = 2
F = 2
C = 3
S_target = 5500
H_FT_target = 800
H_PT_target = 320
O_FT_target = 100

# Define Variables

model = ConcreteModel()
model.S_Dev = Var(within=NonNegativeReals)
model.H_Dev_FT = Var(within=NonNegativeReals)
model.H_Dev_PT = Var(within=NonNegativeReals)
model.O_Dev_FT = Var(within=NonNegativeReals)
model.O_Dev_PT = Var(within=NonNegativeReals)

model.H_FT_Hours = Var(within=NonNegativeReals)
model.H_PT_Hours = Var(within=NonNegativeReals)
model.O_FT_Hours = Var(within=NonNegativeReals)
model.O_PT_Hours = Var(within=NonNegativeReals)

# Define Objective
model.obj = Objective(expr = P1*model.S_Dev + P2*model.O_Dev_FT + P3*F*model.H_Dev_FT +
                      P3*model.H_Dev_PT + P4*model.O_Dev_PT, sense=minimize)

# Define Constraints
model.lower_sales_constraint = Constraint(
    expr= S_target - model.S_Dev <= r_FT * model.H_FT_Hours + r_PT * model.H_PT_Hours)

model.upper_sales_constraint = Constraint(
    expr= r_FT * model.H_FT_Hours + r_PT * model.H_PT_Hours <= S_target + model.S_Dev)

model.lower_full_time_hours_constraint = Constraint(
    expr= H_FT_target - model.H_Dev_FT <= model.H_FT_Hours)

model.upper_full_time_hours_constraint = Constraint(
    expr= model.H_FT_Hours <= H_FT_target + model.H_Dev_FT)

model.lower_part_time_hours_constraint = Constraint(
    expr= H_PT_target - model.H_Dev_PT <= model.H_PT_Hours)

model.upper_part_time_hours_constraint = Constraint(
    expr= model.H_PT_Hours <= H_PT_target + model.H_Dev_PT)

model.lower_full_time_overtime_constraint = Constraint(
    expr= O_FT_target - model.O_Dev_FT <= model.O_FT_Hours)

model.upper_full_time_overtime_constraint = Constraint(
    expr= model.O_FT_Hours <= O_FT_target + model.O_Dev_FT)

model.part_time_overtime = Constraint(expr = model.O_Dev_PT >= model.O_PT_Hours)

# Solve
SolverFactory('glpk').solve(model)

# Display solution
model.pprint()
print(model.obj())

9 Var Declarations
    H_Dev_FT : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 172.0 :  None : False : False : NonNegativeReals
    H_Dev_PT : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    H_FT_Hours : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 972.0 :  None : False : False : NonNegativeReals
    H_PT_Hours : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 320.0 :  None : False : False : NonNegativeReals
    O_Dev_FT : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    O_Dev_PT : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  -0.0 :  None : Fa